# Zenith Advanced Features Validation

**Real-world validation of:**
1. CUDA Graphs
2. Gradient Checkpointing (Phase 1 & 2)
3. Complete E2E Inference

**Important:** This notebook uses REAL tests that verify actual functionality,
not mock tests that always pass.

**Requirements:** GPU Runtime (Runtime > Change runtime type > T4 GPU)

## Setup

In [ ]:
# Install pyzenith v0.2.9 (with memory and inference modules)
!pip install -q pyzenith==0.2.9 torch numpy

# Verify GPU is available
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("WARNING: No GPU detected. CUDA Graphs tests will be skipped.")

In [ ]:
# Import Zenith
import zenith
print(f"Zenith version: {zenith.__version__}")

# Test basic import
from zenith.memory import gradient_checkpointing
from zenith.memory import native_checkpointing
from zenith import inference
print("All modules imported successfully!")

---
## 1. CUDA Graphs Validation

**What we're testing:**
- CUDA Graphs capture and replay
- Performance improvement (should be measurably faster)
- Numerical correctness (outputs must match non-graph execution)

In [ ]:
import time
import numpy as np

def test_cuda_graphs():
    """Test CUDA Graphs with real performance measurement."""
    
    if not torch.cuda.is_available():
        print("SKIPPED: No GPU available")
        return None
    
    print("=" * 60)
    print("TEST: CUDA Graphs")
    print("=" * 60)
    
    # Create a model that benefits from CUDA Graphs
    model = torch.nn.Sequential(
        torch.nn.Linear(512, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 128),
        torch.nn.ReLU(),
        torch.nn.Linear(128, 64),
    ).cuda()
    
    # Fixed input for CUDA Graphs (shape must be constant)
    x = torch.randn(32, 512, device='cuda')
    
    # Warmup
    for _ in range(10):
        _ = model(x)
    torch.cuda.synchronize()
    
    # --- Baseline: Without CUDA Graphs ---
    num_runs = 100
    torch.cuda.synchronize()
    start = time.perf_counter()
    for _ in range(num_runs):
        output_baseline = model(x)
    torch.cuda.synchronize()
    baseline_time = (time.perf_counter() - start) * 1000  # ms
    
    print(f"\nBaseline (no CUDA Graphs): {baseline_time:.2f}ms for {num_runs} runs")
    print(f"  Per-iteration: {baseline_time/num_runs:.3f}ms")
    
    # --- With CUDA Graphs ---
    # Capture graph
    s = torch.cuda.Stream()
    s.wait_stream(torch.cuda.current_stream())
    
    with torch.cuda.stream(s):
        # Warmup in capture stream
        for _ in range(3):
            _ = model(x)
    torch.cuda.current_stream().wait_stream(s)
    
    # Capture
    g = torch.cuda.CUDAGraph()
    static_input = x.clone()
    
    with torch.cuda.graph(g):
        static_output = model(static_input)
    
    # Replay timing
    torch.cuda.synchronize()
    start = time.perf_counter()
    for _ in range(num_runs):
        g.replay()
    torch.cuda.synchronize()
    graph_time = (time.perf_counter() - start) * 1000  # ms
    
    print(f"\nCUDA Graphs: {graph_time:.2f}ms for {num_runs} runs")
    print(f"  Per-iteration: {graph_time/num_runs:.3f}ms")
    
    # --- Verify Correctness ---
    # Run once more with fresh data
    static_input.copy_(x)
    g.replay()
    torch.cuda.synchronize()
    
    output_graph = static_output.clone()
    output_baseline = model(x)
    
    max_diff = (output_graph - output_baseline).abs().max().item()
    
    print(f"\n--- Verification ---")
    print(f"Max difference between graph and baseline: {max_diff:.2e}")
    
    # Results
    speedup = baseline_time / graph_time
    print(f"\n--- Results ---")
    print(f"Speedup: {speedup:.2f}x")
    
    # Assertions
    if max_diff < 1e-5:
        print("NUMERICAL CORRECTNESS: PASSED")
    else:
        print(f"NUMERICAL CORRECTNESS: FAILED (diff={max_diff})")
        return False
    
    if speedup > 1.0:
        print(f"PERFORMANCE: PASSED (speedup={speedup:.2f}x)")
    else:
        print(f"PERFORMANCE: MARGINAL (speedup={speedup:.2f}x)")
    
    return True

cuda_graphs_passed = test_cuda_graphs()

---
## 2. Gradient Checkpointing Phase 1 (PyTorch-based)

**What we're testing:**
- Memory reduction during training
- Gradient correctness (must match non-checkpointed version)

In [ ]:
from zenith.memory.gradient_checkpointing import checkpoint, checkpoint_sequential

def test_gradient_checkpointing_phase1():
    """Test Phase 1 Gradient Checkpointing with real gradient verification."""
    
    print("=" * 60)
    print("TEST: Gradient Checkpointing Phase 1")
    print("=" * 60)
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Create a model with multiple layers
    class DeepModel(torch.nn.Module):
        def __init__(self):
            super().__init__()
            self.layers = torch.nn.ModuleList([
                torch.nn.Linear(256, 256) for _ in range(8)
            ])
            self.activation = torch.nn.ReLU()
        
        def forward(self, x, use_checkpoint=False):
            for layer in self.layers:
                if use_checkpoint:
                    x = checkpoint(lambda inp: self.activation(layer(inp)), x)
                else:
                    x = self.activation(layer(x))
            return x
    
    model = DeepModel().to(device)
    
    # Test input (same for both runs)
    torch.manual_seed(42)
    x = torch.randn(16, 256, device=device, requires_grad=True)
    
    # --- Run WITHOUT checkpointing ---
    x_no_ckpt = x.clone().detach().requires_grad_(True)
    output_no_ckpt = model(x_no_ckpt, use_checkpoint=False)
    loss_no_ckpt = output_no_ckpt.sum()
    loss_no_ckpt.backward()
    grad_no_ckpt = x_no_ckpt.grad.clone()
    
    # --- Run WITH checkpointing ---
    model.zero_grad()
    x_ckpt = x.clone().detach().requires_grad_(True)
    output_ckpt = model(x_ckpt, use_checkpoint=True)
    loss_ckpt = output_ckpt.sum()
    loss_ckpt.backward()
    grad_ckpt = x_ckpt.grad.clone()
    
    # --- Verify Gradient Correctness ---
    grad_diff = (grad_no_ckpt - grad_ckpt).abs().max().item()
    
    print(f"\n--- Gradient Verification ---")
    print(f"Max gradient difference: {grad_diff:.2e}")
    
    if grad_diff < 1e-5:
        print("GRADIENT CORRECTNESS: PASSED")
        return True
    else:
        print(f"GRADIENT CORRECTNESS: FAILED (diff={grad_diff})")
        return False

phase1_passed = test_gradient_checkpointing_phase1()

---
## 3. Gradient Checkpointing Phase 2 (Native Implementation)

**What we're testing:**
- Native checkpointing with activation store
- Optimal checkpoint selection (DP algorithm)
- Memory tracking accuracy

In [ ]:
from zenith.memory.native_checkpointing import (
    native_checkpoint,
    native_checkpoint_sequential,
    NativeCheckpointer,
    OptimalCheckpointSelector,
    ActivationStore
)

def test_native_checkpointing():
    """Test Phase 2 Native Gradient Checkpointing."""
    
    print("=" * 60)
    print("TEST: Gradient Checkpointing Phase 2 (Native)")
    print("=" * 60)
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    all_passed = True
    
    # --- Test 1: Basic native_checkpoint ---
    print("\n[1] Testing native_checkpoint function...")
    
    def simple_fn(x):
        return x * 2 + 1
    
    x = torch.randn(4, 4, device=device, requires_grad=True)
    
    # Without checkpoint
    x1 = x.clone().detach().requires_grad_(True)
    out1 = simple_fn(x1)
    out1.sum().backward()
    grad1 = x1.grad.clone()
    
    # With native checkpoint
    x2 = x.clone().detach().requires_grad_(True)
    out2 = native_checkpoint(simple_fn, x2)
    out2.sum().backward()
    grad2 = x2.grad.clone()
    
    diff = (grad1 - grad2).abs().max().item()
    if diff < 1e-6:
        print(f"  PASSED (gradient diff: {diff:.2e})")
    else:
        print(f"  FAILED (gradient diff: {diff:.2e})")
        all_passed = False
    
    # --- Test 2: Sequential checkpointing ---
    print("\n[2] Testing native_checkpoint_sequential...")
    
    layers = torch.nn.ModuleList([
        torch.nn.Linear(64, 64) for _ in range(4)
    ]).to(device)
    
    x = torch.randn(8, 64, device=device, requires_grad=True)
    
    # Without checkpoint
    x1 = x.clone().detach().requires_grad_(True)
    out1 = x1
    for layer in layers:
        out1 = layer(out1)
    out1.sum().backward()
    grad1 = x1.grad.clone()
    
    # Reset grads
    for layer in layers:
        if layer.weight.grad is not None:
            layer.weight.grad.zero_()
            layer.bias.grad.zero_()
    
    # With sequential checkpoint - use keyword args correctly
    x2 = x.clone().detach().requires_grad_(True)
    out2 = native_checkpoint_sequential(
        functions=list(layers),
        segments=2,
        input_tensor=x2
    )
    out2.sum().backward()
    grad2 = x2.grad.clone()
    
    diff = (grad1 - grad2).abs().max().item()
    if diff < 1e-4:
        print(f"  PASSED (gradient diff: {diff:.2e})")
    else:
        print(f"  FAILED (gradient diff: {diff:.2e})")
        all_passed = False
    
    # --- Test 3: OptimalCheckpointSelector ---
    print("\n[3] Testing OptimalCheckpointSelector (DP algorithm)...")
    
    selector = OptimalCheckpointSelector(
        num_layers=10,
        memory_costs=[1.0] * 10,
        compute_costs=[1.0] * 10,
    )
    
    # Test sqrt heuristic
    ckpts_sqrt = selector.select_checkpoints_sqrt()
    print(f"  Sqrt heuristic: {len(ckpts_sqrt)} checkpoints at {ckpts_sqrt}")
    
    # Test DP algorithm (no arguments)
    ckpts_dp = selector.select_checkpoints_dp()
    print(f"  DP algorithm: {len(ckpts_dp)} checkpoints at {ckpts_dp}")
    
    if len(ckpts_sqrt) > 0 and len(ckpts_dp) > 0:
        print("  PASSED")
    else:
        print("  WARNING: No checkpoints selected")
    
    # --- Test 4: ActivationStore ---
    print("\n[4] Testing ActivationStore...")
    
    # max_memory_bytes (10 MB = 10 * 1024 * 1024 bytes)
    store = ActivationStore(max_memory_bytes=10 * 1024 * 1024)
    
    # Store some tensors
    t1 = torch.randn(100, 100, device=device)
    t2 = torch.randn(100, 100, device=device)
    
    store.store(0, t1)  # layer_id = 0
    store.store(1, t2)  # layer_id = 1
    
    # Retrieve
    r1 = store.get(0)
    r2 = store.get(1)
    
    if r1 is not None and r2 is not None:
        if torch.allclose(t1, r1) and torch.allclose(t2, r2):
            print("  PASSED (store/retrieve works correctly)")
        else:
            print("  FAILED (data mismatch)")
            all_passed = False
    else:
        print("  FAILED (tensors not found)")
        all_passed = False
    
    print(f"\n--- Summary ---")
    if all_passed:
        print("ALL NATIVE CHECKPOINTING TESTS PASSED")
    else:
        print("SOME TESTS FAILED")
    
    return all_passed

phase2_passed = test_native_checkpointing()

---
## 4. Complete E2E Inference

**What we're testing:**
- InferenceSession creation and execution
- Benchmark API accuracy
- Output correctness (must match PyTorch native output)

In [ ]:
from zenith.inference import (
    InferenceSession,
    InferenceConfig,
    InferenceStats,
    InferenceResult,
    create_session,
    infer
)

def test_e2e_inference():
    """Test Complete E2E Inference Pipeline."""
    
    print("=" * 60)
    print("TEST: Complete E2E Inference")
    print("=" * 60)
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    all_passed = True
    
    # --- Test 1: Basic Session Creation ---
    print("\n[1] Testing InferenceSession creation...")
    
    model = torch.nn.Sequential(
        torch.nn.Linear(128, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 32),
    ).to(device)
    
    sample_input = torch.randn(4, 128, device=device)
    
    config = InferenceConfig(
        backend=device,
        verbose=0,
        warmup_iterations=2
    )
    
    session = InferenceSession(model, config=config, sample_input=sample_input)
    
    if session.is_initialized:
        print(f"  Session created: framework={session.framework}, backend={session.backend}")
        print("  PASSED")
    else:
        print("  FAILED: Session not initialized")
        all_passed = False
    
    # --- Test 2: Inference Run ---
    print("\n[2] Testing inference run...")
    
    test_input = torch.randn(4, 128, device=device)
    
    # Run via session
    result = session.run({'input': test_input})
    
    # Run via native PyTorch
    with torch.no_grad():
        expected = model(test_input)
    
    # Compare
    if isinstance(result, dict):
        out_tensor = list(result.values())[0]
        if isinstance(out_tensor, np.ndarray):
            out_tensor = torch.from_numpy(out_tensor).to(device)
        elif hasattr(out_tensor, 'cpu'):
            out_tensor = out_tensor.to(device)
    else:
        out_tensor = result
    
    diff = (out_tensor.float() - expected.float()).abs().max().item()
    
    if diff < 1e-4:
        print(f"  Output matches native PyTorch (diff: {diff:.2e})")
        print("  PASSED")
    else:
        print(f"  FAILED: Output mismatch (diff: {diff:.2e})")
        all_passed = False
    
    # --- Test 3: Latency Return ---
    print("\n[3] Testing latency measurement...")
    
    result_with_latency = session.run({'input': test_input}, return_latency=True)
    
    if isinstance(result_with_latency, InferenceResult):
        print(f"  Latency: {result_with_latency.latency_ms:.3f}ms")
        print(f"  Backend: {result_with_latency.backend_used}")
        print("  PASSED")
    else:
        print("  FAILED: Expected InferenceResult")
        all_passed = False
    
    # --- Test 4: Statistics ---
    print("\n[4] Testing statistics tracking...")
    
    # Run 10 more times
    for _ in range(10):
        session.run({'input': test_input})
    
    stats = session.get_stats()
    
    print(f"  Total runs: {stats['total_runs']}")
    print(f"  Mean latency: {stats['mean_latency_ms']:.3f}ms")
    print(f"  Min latency: {stats['min_latency_ms']:.3f}ms")
    print(f"  Max latency: {stats['max_latency_ms']:.3f}ms")
    
    if stats['total_runs'] >= 10 and stats['mean_latency_ms'] > 0:
        print("  PASSED")
    else:
        print("  FAILED")
        all_passed = False
    
    # --- Test 5: Benchmark API ---
    print("\n[5] Testing benchmark API...")
    
    bench = session.benchmark({'input': test_input}, num_runs=20, num_warmup=5)
    
    print(f"  Benchmark results:")
    print(f"    Mean: {bench['mean_ms']:.3f}ms")
    print(f"    P50:  {bench['p50_ms']:.3f}ms")
    print(f"    P99:  {bench['p99_ms']:.3f}ms")
    print(f"    Throughput: {bench['throughput_per_sec']:.1f}/sec")
    
    required_keys = ['mean_ms', 'std_ms', 'min_ms', 'max_ms', 'p50_ms', 'p90_ms', 'p99_ms']
    if all(k in bench for k in required_keys):
        print("  PASSED")
    else:
        print("  FAILED: Missing benchmark keys")
        all_passed = False
    
    # --- Test 6: Convenience Functions ---
    print("\n[6] Testing convenience functions...")
    
    # create_session
    session2 = create_session(model, config=config, sample_input=sample_input)
    if session2.is_initialized:
        print("  create_session: PASSED")
    else:
        print("  create_session: FAILED")
        all_passed = False
    
    # infer (one-shot)
    result = infer(model, {'input': test_input}, config=config)
    if result is not None and len(result) > 0:
        print("  infer: PASSED")
    else:
        print("  infer: FAILED")
        all_passed = False
    
    print(f"\n--- Summary ---")
    if all_passed:
        print("ALL E2E INFERENCE TESTS PASSED")
    else:
        print("SOME TESTS FAILED")
    
    return all_passed

e2e_passed = test_e2e_inference()

---
## Final Summary

In [ ]:
print("=" * 60)
print("ZENITH ADVANCED FEATURES - VALIDATION SUMMARY")
print("=" * 60)

results = {
    "CUDA Graphs": cuda_graphs_passed,
    "Gradient Checkpointing Phase 1": phase1_passed,
    "Gradient Checkpointing Phase 2 (Native)": phase2_passed,
    "Complete E2E Inference": e2e_passed,
}

for name, passed in results.items():
    if passed is None:
        status = "SKIPPED"
    elif passed:
        status = "PASSED"
    else:
        status = "FAILED"
    print(f"{name}: {status}")

total_passed = sum(1 for p in results.values() if p is True)
total_failed = sum(1 for p in results.values() if p is False)
total_skipped = sum(1 for p in results.values() if p is None)

print()
print(f"Total: {total_passed} passed, {total_failed} failed, {total_skipped} skipped")

if total_failed == 0:
    print("\n*** ALL VALIDATIONS SUCCESSFUL ***")
else:
    print("\n*** SOME VALIDATIONS FAILED - NEEDS INVESTIGATION ***")